In [1]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import re
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)

In [3]:
import pandas as pd
df = pd.read_csv('train.csv')
df = df.iloc[0:500,0]
df

0      CHAPTER IV. \n\nNotwithstanding the earnest in...
1      (CNN) -- Polk County, Florida, detectives arre...
2      Malawi (, or ; or [maláwi]), officially the Re...
3      CHAPTER XXXII. \n\nMR. GILMORE'S SUCCESS. \n\n...
4      Computer security, also known as cybersecurity...
                             ...                        
495    CHAPTER VII \n\nBut, ah me! she's a heart of s...
496    North America is a continent entirely within t...
497    Memphis is a city in the southwestern corner o...
498    Biodiversity, a contraction of "biological div...
499    Paul the pear lived in a bowl on the table. He...
Name: Story, Length: 500, dtype: object

In [4]:
df = pd.DataFrame(df)
df

,Story
0,CHAPTER IV. \n\nNotwithstanding the earnest in...
1,"(CNN) -- Polk County, Florida, detectives arre..."
2,"Malawi (, or ; or [maláwi]), officially the Re..."
3,CHAPTER XXXII. \n\nMR. GILMORE'S SUCCESS. \n\n...
4,"Computer security, also known as cybersecurity..."
...,...
495,"CHAPTER VII \n\nBut, ah me! she's a heart of s..."
496,North America is a continent entirely within t...
497,Memphis is a city in the southwestern corner o...
498,"Biodiversity, a contraction of ""biological div..."


In [5]:
import re
# Lowercasing the text
df['cleaned']=df['Story'].apply(lambda x:x.lower())

# Dictionary of english Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","shed": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who' will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Expanding Contractions
df['cleaned']=df['cleaned'].apply(lambda x:expand_contractions(x))

In [6]:
df.head(5)

,Story,cleaned
0,CHAPTER IV. \n\nNotwithstanding the earnest in...,chapter iv. \n\nnotwithstanding the earnest in...
1,"(CNN) -- Polk County, Florida, detectives arre...","(cnn) -- polk county, florida, detectives arre..."
2,"Malawi (, or ; or [maláwi]), officially the Re...","malawi (, or ; or [maláwi]), officially the re..."
3,CHAPTER XXXII. \n\nMR. GILMORE'S SUCCESS. \n\n...,chapter xxxii. \n\nmr. gilmore is success. \n\...
4,"Computer security, also known as cybersecurity...","computer security, also known as cybersecurity..."


In [7]:
def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text
 
# Cleaning corpus using RegEx
df['cleaned']=df['cleaned'].apply(lambda x: clean_text(x))

In [8]:
df.drop("Story",axis = 1, inplace = True)
df

,cleaned
0,chapter iv notwithstanding the earnest inju...
1,cnn polk county florida detectives arre...
2,malawi or or mal wi officially the re...
3,chapter xxxii mr gilmore is success har...
4,computer security also known as cybersecurity...
...,...
495,chapter vii but ah me she is a heart of st...
496,north america is a continent entirely within t...
497,memphis is a city in the southwestern corner o...
498,biodiversity a contraction of biological div...


In [9]:
# Function to generate embeddings
def embeddings(text):
    return np.array(model(text)).flatten().tolist()

In [13]:
print(embeddings(['jf']))
len(embeddings(['jf']))

[0.028594959527254105, 0.062198441475629807, 0.10926178097724915, -0.010224954225122929, 0.014060011133551598, -0.04632542282342911, 0.07072386890649796, -0.06861643493175507, -0.08257300406694412, 0.0005479017272591591, -0.020785802975296974, 0.05393010005354881, 0.0385376513004303, -0.04703380912542343, -0.04125058650970459, -0.0327012725174427, -0.0403803326189518, -0.0462595596909523, -0.00848297867923975, 0.07183258980512619, 0.030858321115374565, 0.04788857698440552, 0.03722039982676506, -0.05101475864648819, 0.006195284426212311, 0.012189959175884724, 0.06906178593635559, 0.02678343839943409, 0.04782657325267792, -0.01295535545796156, 0.0023022773675620556, -0.01879492960870266, -0.0128092672675848, 0.07272717356681824, -0.05806437507271767, -0.06611934304237366, 0.00983378104865551, -0.0024549232330173254, -0.03866569697856903, -0.03217332810163498, 0.012122951447963715, 0.0575065053999424, -0.018025750294327736, -0.05521390959620476, 0.06227634847164154, 0.0031091044656932354,

In [14]:
# with open(file_path) as file:
#     lines = [line for line in file]
# lines = df.to_string(index=False, header=False).split('\n')
# msgs = [x.split('\t')[1].replace('\n', '')   for x in lines]
# embdngs = [embeddings([x]) for x in msgs]
# indx = list(range(1, len(msgs)+1))
# data_to_insert = [indx, msgs, embdngs]

# msgs = df.iloc[:, 0].tolist()
# embdngs = [embeddings(x) for x in msgs]
# indx = list(range(1, len(msgs) + 1))
# data_to_insert = [indx, msgs, embdngs]

msgs = df.iloc[:, 0].tolist()
embdngs = [embeddings([x]) for x in msgs]
indx = list(range(1, len(msgs) + 1))
data_to_insert = [indx, msgs, embdngs]


In [15]:
connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

In [16]:
# Field Schema
id = FieldSchema(
  name="id",
  dtype=DataType.INT64,
  is_primary=True,
)
message = FieldSchema(
  name="message",
  dtype=DataType.VARCHAR,
  max_length=6000,
)
message_vec = FieldSchema(
  name="message_embeddings",
  dtype=DataType.FLOAT_VECTOR,
  dim=512
)
# collection schema
collection_schema = CollectionSchema(
  fields=[id, message, message_vec],
  description="Similar Document Retreival"
)
# Create collection
collection = Collection(
    name="SDR",
    schema=collection_schema,
    using='default')
utility.list_collections()

['Album1', 'Spam_Test', 'SDR']

In [17]:
data_insert = collection.insert(data_to_insert)

In [18]:
# Create Index
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024},
  "index_name": "SDR_IVF_FLAT_TEST"
}

# Index on vector field
collection.create_index(
  field_name="message_embeddings",
  index_params=index_params
)

Status(code=0, message=)

In [19]:
# Load the collection

collection.load(replica_number=1)

In [28]:
# test message
test_message = ["Golden Retriver"]
test_message_vector = embeddings(test_message)

In [45]:
## Vector Similarity Search
search_params = {"metric_type": "L2", "params": {"nprobe": 64}}

results = collection.search(
	data=[test_message_vector],
	anns_field="message_embeddings",
	param=search_params,
	limit=10,
	expr=None,
	output_fields=['message'],
)

results

KeyboardInterrupt: 